In [63]:
# Wikipedia Tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [64]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_limit=100)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [65]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model

In [66]:
llm = init_chat_model(
    "qwen2.5-coder:0.5b",
    model_provider="ollama",
)

In [67]:
loader = WebBaseLoader("https://www.freecodecamp.org/news/multithreading-for-beginners/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [68]:
vector_store = FAISS.from_documents(
    documents=texts,
    embedding=OllamaEmbeddings(model="qwen2.5-coder:0.5b"))

In [69]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10cdefed0>, search_kwargs={})

In [70]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="DocumentRetriever",
    description="Useful for when you need to look up documents related to multithreading in Python."
)

In [71]:
retriever_tool.name

'DocumentRetriever'

In [72]:
# Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_limit=100)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [73]:
arxiv_tool.name

'arxiv'

In [74]:
tools = [wiki_tool, arxiv_tool, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/toshalimohapatra/Documents/Chatbot_Langchain/venv/lib/python3.13/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='DocumentRetriever', description='Useful for when you need to look up documents related to multithreading in Python.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10e252980>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_

In [75]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("hwchase17/react")   # or any other hub handle

In [76]:
system_prompt = getattr(prompt, "template", str(prompt))

In [78]:
from langchain_classic import hub
from langchain_classic.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool   

@tool
def retriever_tool(query: str) -> str:
    """Search my local ML docs and return relevant chunks."""
    docs = retriever.get_relevant_documents(query)
    return "\n\n".join(d.page_content for d in docs[:2])

prompt = hub.pull("hwchase17/react")  

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,              
    handle_parsing_errors=True 
)

response = agent_executor.invoke({"input": "How does multithreading affect CPU performance?"})
print(response["output"])





> Entering new AgentExecutor chain...
Action: DocumentRetriever
Action Input: "document_prompt", "document_separator"Multithreading for Beginners





































































            Search
        





Submit your search query










Forum
Donate














                            July 16, 2024
                        
/

                                #multithreading
                            

Multithreading for Beginners







                    
                        Beau Carnes

Front-End Libraries
                    


Express and Node.js
                    


Python Code Examples
                    


Clustering in Python
                    


Software Architecture
                    


Programming Fundamentals
                    


Coding Career Preparation
                    


Full-Stack Developer Guide
                    


Python for JavaScript Devs
                    





                    Mobile 